**搭建模型**

In [35]:
from transformers import BertModel
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
from tqdm import tqdm
import os
import transformers
from transformers import BertTokenizer
import torch
import numpy as np

In [36]:
class BertClassifier(nn.Module):
    def __init__(self, model_path, dropout=0.5,freeze_bert=True):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_path)
        self.dropout = nn.Dropout(dropout)
        self.linear1 = nn.Linear(768, 768)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(768, 2)
        self.sig = nn.Sigmoid()

        # 冻结BERT模型的参数
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear1_output = self.relu(self.linear1(dropout_output))
        linear2_output = self.linear2(dropout_output)
        final_layer = self.sig(linear2_output)
        return final_layer

In [37]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, datas, model_path):
        self.labels = datas['label']
        tokenizer = BertTokenizer.from_pretrained(model_path)
        self.reviews = [
            tokenizer(str(review), padding='max_length', max_length=512, truncation=True, return_tensors='pt')
            for review in datas['review']
        ]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.reviews[idx], np.array(self.labels[idx])

In [38]:
def train(model, train_dataset, val_dataset, epochs):
    train_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=24)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optim = transformers.AdamW(model.parameters(), lr=1e-5,correct_bias=True)
    best_avg_acc_val = 0

    for epoch in range(epochs):
        model.train()
        total_loss_train = total_acc_train = 0
        train_iterator = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs} [Train]', leave=False)
        for train_input, train_label in train_iterator:
            train_label = train_label.to(device)
            attention_mask = train_input['attention_mask'].squeeze(1).to(device)
            input_ids = train_input['input_ids'].squeeze(1).to(device)
            output = model(input_ids, attention_mask)
            loss = criterion(output, train_label)
            total_loss_train += loss.item()
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc
            model.zero_grad()
            loss.backward()
            optim.step()

        model.eval()
        total_loss_val = total_acc_val = 0
        val_iterator = tqdm(val_loader, desc=f'Epoch {epoch + 1}/{epochs} [Validate]', leave=False)
        with torch.no_grad():
            for val_input, val_label in val_iterator:
                val_label = val_label.to(device)
                attention_mask = val_input['attention_mask'].squeeze(1).to(device)
                input_ids = val_input['input_ids'].squeeze(1).to(device)
                output = model(input_ids, attention_mask)
                loss = criterion(output, val_label)
                total_loss_val += loss.item()
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        avg_acc_val = total_acc_val / len(val_dataset)
        if avg_acc_val > best_avg_acc_val:
            best_avg_acc_val = avg_acc_val
            save_path = "/content/model"
            directory = os.path.dirname(save_path)
            # 如果目录不存在，创建它
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save(model.state_dict(), save_path)

        print(f'Epoch {epoch + 1}: Train Loss {total_loss_train / len(train_dataset):.3f}, '
              f'Train Acc {total_acc_train / len(train_dataset):.3f}, '
              f'Val Loss {total_loss_val / len(val_dataset):.3f}, '
              f'Val Acc {avg_acc_val:.3f}')

In [39]:
train_dataset = pd.read_excel('/content/data/train_dataset.xlsx')
dataset = Dataset(train_dataset, "bert-base-chinese")
train_data, val_data = random_split(dataset, [round(0.8 * len(train_dataset)), round(0.2 * len(train_dataset))])
model = BertClassifier("bert-base-chinese",0.5,freeze_bert=True)
print('训练开始')
train(model,train_dataset=train_data,val_dataset=val_data,epochs=10)
print('训练结束')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


训练开始


Epoch 1: Train Loss 0.027, Train Acc 0.645, Val Loss 0.027, Val Acc 0.690


Epoch 2: Train Loss 0.026, Train Acc 0.689, Val Loss 0.026, Val Acc 0.690


Epoch 3: Train Loss 0.026, Train Acc 0.688, Val Loss 0.026, Val Acc 0.690


...

Epoch 6: Train Loss 0.026, Train Acc 0.687, Val Loss 0.025, Val Acc 0.690


Epoch 7: Train Loss 0.025, Train Acc 0.687, Val Loss 0.025, Val Acc 0.690


Epoch 8: Train Loss 0.025, Train Acc 0.688, Val Loss 0.025, Val Acc 0.690


Epoch 9: Train Loss 0.025, Train Acc 0.688, Val Loss 0.025, Val Acc 0.690


Epoch 10: Train Loss 0.025, Train Acc 0.689, Val Loss 0.025, Val Acc 0.690
训练结束


第一回合：

Epoch 1: Train Loss 0.029, Train Acc 0.477, Val Loss 0.027, Val Acc 0.
721

Epoch 2: Train Loss 0.027, Train Acc 0.675, Val Loss 0.026, Val Acc 0.706

Epoch 3: Train Loss 0.026, Train Acc 0.686, Val Loss 0.025, Val Acc 0.705

Epoch 4: Train Loss 0.026, Train Acc 0.686, Val Loss 0.025, Val Acc 0.705

第二回合：

Epoch 1: Train Loss 0.027, Train Acc 0.645, Val Loss 0.027, Val Acc 0.690

Epoch 2: Train Loss 0.026, Train Acc 0.689, Val Loss 0.026, Val Acc 0.690

Epoch 3: Train Loss 0.026, Train Acc 0.688, Val Loss 0.026, Val Acc 0.690

Epoch 4: Train Loss 0.026, Train Acc 0.687, Val Loss 0.026, Val Acc 0.690

